In [1]:
!gdown --id 16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT
!unzip -qq ECG5000.zip
!pip install  -qq arff2pandas
from sklearn.metrics import confusion_matrix, classification_report
from glob import glob
import time
import copy
import shutil

import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F
from arff2pandas import a2p


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style="whitegrid",palette='muted',font_scale=1.2)
HAPPY_COLORS_PALETTE=["#01BEFE","#FFDD00","#FF7D00","#FF006D","#ADFF02","#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize']=12,8

RANDOM_SEED =42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open('ECG5000_TRAIN.arff') as f:
  train = a2p.load(f)
with open('ECG5000_TEST.arff') as f:
  test = a2p.load(f)
df = train.append(test)
df = df.sample(frac=1.0)
CLASS_NORMAL = 1
new_columns = list(df.columns)
new_columns[-1] = 'target'
df.columns = new_columns



Downloading...
From: https://drive.google.com/uc?id=16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT
To: /content/ECG5000.zip
100% 10.6M/10.6M [00:00<00:00, 33.4MB/s]


In [2]:
dataset  = df

In [8]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 140].values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [12]:
y_pred = classifier.predict(X_test)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[576   1   1   0   0]
 [  1 345   4   2   0]
 [  5   6   9   2   0]
 [  1  25   0  19   0]
 [  3   0   0   0   0]]
              precision    recall  f1-score   support

           1       0.98      1.00      0.99       578
           2       0.92      0.98      0.95       352
           3       0.64      0.41      0.50        22
           4       0.83      0.42      0.56        45
           5       0.00      0.00      0.00         3

    accuracy                           0.95      1000
   macro avg       0.67      0.56      0.60      1000
weighted avg       0.94      0.95      0.94      1000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(kernel='linear')

In [15]:
y_pred = svclassifier.predict(X_test)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[572   3   2   0   1]
 [  1 340   7   4   0]
 [  1   7  12   2   0]
 [  2  30   1  12   0]
 [  3   0   0   0   0]]
              precision    recall  f1-score   support

           1       0.99      0.99      0.99       578
           2       0.89      0.97      0.93       352
           3       0.55      0.55      0.55        22
           4       0.67      0.27      0.38        45
           5       0.00      0.00      0.00         3

    accuracy                           0.94      1000
   macro avg       0.62      0.55      0.57      1000
weighted avg       0.93      0.94      0.93      1000



SMOTE


In [20]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[('1', 2919), ('2', 2919), ('3', 2919), ('4', 2919), ('5', 2919)]


In [21]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_resampled, y_resampled)

SVC(kernel='linear')

In [22]:
y_pred = svclassifier.predict(X_test)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[447  29  87  11   4]
 [  4 318  18  11   1]
 [  1   6  15   0   0]
 [  2  37   2   4   0]
 [  2   0   0   0   1]]
              precision    recall  f1-score   support

           1       0.98      0.77      0.86       578
           2       0.82      0.90      0.86       352
           3       0.12      0.68      0.21        22
           4       0.15      0.09      0.11        45
           5       0.17      0.33      0.22         3

    accuracy                           0.79      1000
   macro avg       0.45      0.56      0.45      1000
weighted avg       0.86      0.79      0.81      1000



In [24]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_resampled, y_resampled)

KNeighborsClassifier()

In [25]:
y_pred = classifier.predict(X_test)

In [26]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[468  31  69   6   4]
 [  1 275  23  25  28]
 [  0   5  14   3   0]
 [  0  17   7  20   1]
 [  1   0   2   0   0]]
              precision    recall  f1-score   support

           1       1.00      0.81      0.89       578
           2       0.84      0.78      0.81       352
           3       0.12      0.64      0.20        22
           4       0.37      0.44      0.40        45
           5       0.00      0.00      0.00         3

    accuracy                           0.78      1000
   macro avg       0.47      0.53      0.46      1000
weighted avg       0.89      0.78      0.82      1000

